# Simplest use case

## Running a model from the config file

Setting up the `dolphin` ecosystem:

- Install `dolphin`, `lenstronomy`, and the required dependencies.
- Create an input/output directory for `dolphin`, we are using "../io_directory_example" in this example.
- Set up these directories inside the input/output directory. Look inside "../io_directory_example" for example. 
    - **data**: contains subdirectories for each lens system with a data and PSF files for each band.
    - **settings**: contains the 'config_{lens_name}.yml' files for each lens system,
    - **logs**: to write the log files from model runs,
    - **outputs**: to save the model outputs,
    - **hpc**: *optional*, contains scripts to submit batch jobs in MPI.
 

   
### Data format

The image data file needs to be in the hdf5 formate with the following datasets:

- `image_data`: reduced and background subtracted image cutout centered at the lens system,
- `background_rms`: background level,
- `exposure_time`: the map of exposure times for each pixel, so that `image_data * exposure_time` is Poisson noise distributed,
- `ra_at_xy_0`: RA of the (0, 0) pixel in the `image_data` cutout,
- `dec_at_xy_0`: Dec of the (0, 0) pixel in the `image_data` cutout,
- `transform_pix2angle`: a transform matrix to map the pixel numbers (x, y) to angles (RA, Dec).

The PSF data file needs to be in the hdf5 format with the following datasets:

- `kernel_point_source`: a pixelated PSF (not required to have the same dimension of `image_data`),
- `psf_error_map`: *optional*, uncertainty in the provided PSF, needs to have the same dimension of `kernel_point_source`. 

### Config file format

The 'config_{lens_name}.yml' file provides you options to set up the lens model. Here is the content of the config file that is used in the example model below.

### create a `Processor` instance and point to the IO directory

In [1]:
# from dolphin.processor import Processor

# processor = Processor('../io_directory_example/')

### Run a model by calling the  `swim()` method

In [2]:
# processor.swim(lens_name='DESJ0537-4647', model_id='example', log=False)

## Let's check the output

In [3]:
from dolphin.analysis.output import Output

output = Output('../io_directory_example/')

In [8]:
fig = output.plot_model_overview(lens_name='DESJ0537-4647', model_id='F140W_Run01')

IndexError: boolean index did not match indexed array along dimension 0; dimension is 19600 but corresponding boolean dimension is 7744

We only ran a pre-sampling optimization here and did not perform a MCMC sampling. So, the above model is the optimized model that the MCMC sample can initiate from. The `kwargs_result` dictionary of the pre-sampling optimization step can be accessed through `Output.kwargs_result` after loading an output using the `Output.load_output()` method.

In [5]:
output.load_output(lens_name='DESJ0537-4647', model_id='F140W_Run01')

output.kwargs_result

{'kwargs_lens': [{'theta_E': 1.993402210977581,
   'gamma': 2.0,
   'e1': -0.2155160936437417,
   'e2': -0.044826373357606406,
   'center_x': -0.003495513553335139,
   'center_y': 0.011476362249842897},
  {'gamma_ext': 0.001, 'psi_ext': 0.0, 'ra_0': 0, 'dec_0': 0}],
 'kwargs_source': [{'amp': 1,
   'n_max': 6,
   'beta': 0.13633238309699788,
   'center_x': -0.6420948518655116,
   'center_y': -0.04549991473155932},
  {'amp': 1,
   'R_sersic': 0.2383504761257381,
   'n_sersic': 4.0,
   'e1': 0.4999983801387821,
   'e2': 0.057534724817228496,
   'center_x': -0.6420948518655116,
   'center_y': -0.04549991473155932}],
 'kwargs_lens_light': [{'amp': 1,
   'R_sersic': 0.753825412620007,
   'n_sersic': 4.0,
   'e1': -0.13410888851815445,
   'e2': 0.01090414742079265,
   'center_x': -0.003495513553335139,
   'center_y': 0.011476362249842897},
  {'amp': 1,
   'R_sersic': 3.1631283578619054,
   'n_sersic': 1.0,
   'e1': -0.13410888851815445,
   'e2': 0.01090414742079265,
   'center_x': -0.0034955

When necessary, the settings of the model run---that was read out of the 'config_{lens_name}.yml' file---can be accessed through the `output.model_settings` variable.

In [6]:
output.model_settings

{'band': ['F140W'],
 'fitting': {'mcmc_settings': {'burnin_step': 0,
   'iteration_step': 2000,
   'walker_ratio': 16},
  'psf_iteration': False,
  'psf_iteration_settings': {'block_neighbor': 0.0,
   'psf_iteration_factor': 0.5,
   'psf_iteration_num': 50,
   'psf_symmetry': 4},
  'pso': True,
  'pso_settings': {'num_iteration': 248, 'num_particle': 250},
  'sampler': 'MCMC',
  'sampling': False},
 'guess_params': {'lens': {'0': {'theta_E': 1.9965223041493796}}},
 'kwargs_constraints': {'joint_lens_light_with_lens_light': [[0,
    1,
    ['e1', 'e2', 'center_x', 'center_y']]],
  'joint_lens_with_light': [[0, 0, ['center_x', 'center_y']]],
  'joint_source_with_source': [[0, 1, ['center_x', 'center_y']]]},
 'lens_light_option': {'fix': {'0': {'n_sersic': 4.0},
   '1': {'n_sersic': 1.0}}},
 'lens_option': {'centroid_init': [0.0, 0.0],
  'constrain_position_angle_from_lens_light': 15},
 'mask': {'centroid_offset': [[0.04910000000000003, 0.1451]],
  'custom_mask': [None],
  'dec_at_xy_0': 